# Gaussian curvature approximation on Riemannian manifolds

In this tutorial we compute the Gaussian curvature of a planar domain equipped with a non-Euclidean metric.

Let $\Omega=[-1,1]^2$ and define the metric tensor
$$
g= \left(\begin{array}{cc}
1+(x-\frac{1}{3}x^3)^2 &(x-\frac{1}{3}x^3)(y-\frac{1}{3}y^3)\\(x-\frac{1}{3}x^3)(y-\frac{1}{3}y^3)&1+(y-\frac{1}{3}y^3)^2
\end{array}\right).
$$
This is the metric tensor induced by the embedding of the surface described by the graph
$$
(x,y)\mapsto (x,y,f(x,y)),\qquad\qquad f(x,y) = \frac{1}{2}(x^2+y^2)-\frac{1}{12}(x^4+y^4).
$$
The exact Gaussian curvature is given by
\begin{align*}
K(g)(x,y) = \frac{81(1-x^2)(1-y^2)}{(9+x^2(x^2-3)^2+y^2(y^2-3)^2)^2}.
\end{align*}

In [ ]:
from ngsolve.meshes import MakeStructured2DMesh
from ngsolve import *
from ngsolve.webgui import Draw
import random as random
from ngsolve.krylovspace import CGSolver

mapping = lambda x,y : (-1+2*x, -1+2*y)
mesh = MakeStructured2DMesh(quads=False, nx=8, ny=8, mapping = mapping)

Gex = CF( (1+(x-1/3*x**3)**2,(x-1/3*x**3)*(y-1/3*y**3),(x-1/3*x**3)*(y-1/3*y**3),1+(y-1/3*y**3)**2), dims=(2,2) )
K_ex = 81*(1-x**2)*(1-y**2)/(9+x**2*(x**2-3)**2+y**2*(y**2-3)**2)**2
Draw(K_ex, mesh, "K", order=3, deformation=True)

First we define a function that computes the $H^{-1}$-norm of a functional $f$, using the fact that $||f||_{H^{-1}}$ is equivalent to $\|u\|_{H^1}$ if $-\Delta u = f$.  We will use this function later on to compute the error in the discrete Gaussian curvature.

In [ ]:
#H^-1 norm
def CompHm1Norm(rhs, order):
    fesH = H1(mesh, order=order, dirichlet=".*")
    u,v = fesH.TnT()
    
    a = BilinearForm(Grad(u)*Grad(v)*dx, symmetric=True, symmetric_storage=True, condense=True)
    f = LinearForm(rhs*v*dx).Assemble()
    
    apre = Preconditioner(a, "bddc")
    a.Assemble()
    invS = CGSolver(a.mat, apre.mat, printrates='\r', maxiter=400)
    ext = IdentityMatrix()+a.harmonic_extension
    inv = a.inner_solve + ext @ invS @ ext.T
    
    gfu = GridFunction(fesH)
    gfu.vec.data = inv*f.vec
    
    err = sqrt( Integrate( gfu**2 + Grad(gfu)**2, mesh) )
    return err

Next we define a function for computing lifted Gaussian curvature $K_h\in V_h^{k+1}$ for a Regge metric $g_h\in \mathrm{Reg}^k$ approximating the exact metric tensor $g$; that is, $g_h = \mathcal{I}^k_hg$.

The right-hand side of the lifting is given by
$$
f = \sum_{V \in \mathcal{V}_h} \Theta_V u(V) + \sum_{E \in \mathcal{E}_h} \int_E [\![ \kappa_g ]\!] \,u ds + \sum_{T \in \mathcal{T}_h} \int_T K|_T u dx, \quad u \in V_h^{k+1}.
$$

To compute the Gaussian curvature $K$ and geodesic curvature $\kappa_g$ appearing above, we start by recalling the definition of the Christoffel symbols of second and first kind
$$
\Gamma_{ij}^k(g) 
	=\frac{1}{2}g ^{kl}\left(\partial_{i}g_{jl}+\partial_{j}g_{il}-\partial_{l}g_{ij}\right)=g ^{kl}\Gamma_{ijl}(g).
$$
Here, we used Einstein summation convention of repeated indices and that $g^{kl}$ is the (k,l)-entry of the inverse of $g$.

The full fourth order Riemann curvature tensor encoding all intrinsic curvature information reads in coordinates
$$
\mathrm{Riem}_{ijkl} = 
			\partial_i \Gamma_{jkl} - \partial_j \Gamma_{ikl}
			+ \Gamma_{ik}^p\Gamma_{jpl}
			- \Gamma_{jk}^p\Gamma_{ipl}.
$$
The Gaussian curvature can be defined in terms of sectional curvature. Since we are in 2D, the Riemann curvature tensor has in fact only 1 independent component, and
$$
K =
\frac{\mathrm{Riem}(u, v, v, u)}{ \| u \|_g \| v\|_g - g(u, v)^2} = \frac{\mathrm{Riem}_{1212}}{\det g}.
$$
The (signed) geodesic curvature $\kappa(g) = g(\nabla_{t_g}t_g,n_g)$ is given in coordinates by
$$
\kappa(g)= 
	\frac{\sqrt{\det g}}{ g_{tt}^{3/2}}
	\left(
	\partial_{t}t\cdot n
	+
	\Gamma_{tt}^{n}\right),\qquad\qquad \Gamma_{tt}^n = \Gamma_{ij}^kt^it^jn_k
	.
$$

The function below computes $K_h$ and outputs the errors $||K_h-K||_{H^{-1}}$ and $||K_h-K||_{L^2}$.

In [ ]:
# tangential and normal vector at element edges
t = specialcf.tangential(2)
n = -specialcf.normal(2)
    
# vectors used to compute angles at a vertex
bbndtang  = specialcf.VertexTangentialVectors(2)
bbndtang1 = bbndtang[:,0]
bbndtang2 = bbndtang[:,1]
def TrigAngle(G):
    return acos( G*bbndtang1*bbndtang2 / sqrt(G*bbndtang1*bbndtang1) / sqrt(G*bbndtang2*bbndtang2) )

def ComputeGaussCurv(mesh, Gex, order, can_int=False):
    fesCC = HCurlCurl(mesh, order=order)
    gfG = GridFunction(fesCC)
    if can_int:
        gfG.Set(Gex, dual=True, bonus_intorder=5)
    else:
        gfG.Set(Gex)
    fesH = H1(mesh, order=order+1, dirichlet=".*")
    u,v = fesH.TnT()

    edgecurve = specialcf.EdgeCurvature(2) # nabla_t t
    
    vol_term = 1/sqrt(Det(gfG))*v*gfG.Operator("Riemann")[0,1,0,1]
    bnd_term = sqrt(Det(gfG))/(t*(gfG*t))*(edgecurve*n+gfG.Operator("christoffel2")[t,t,n])
    bbnd_term = (TrigAngle(Id(2))-TrigAngle(gfG))

    f = LinearForm(fesH)
    # angle deficit
    f += v*bbnd_term*dx(element_vb=BBND)
    # jump of geodesic curvature
    f += v*bnd_term*dx(element_boundary=True)
    # Gauss curvature
    f += vol_term*dx
    
    # lifted Gauss curvature
    gfK = GridFunction(fesH)
    
    with TaskManager():
        M = BilinearForm(sqrt(Det(gfG))*u*v*dx, symmetric=True, symmetric_storage=True, condense=True)
        Mpre = Preconditioner(M, "bddc")
        M.Assemble()
        f.Assemble()
        
        
        invS = CGSolver(M.mat, Mpre.mat, printrates='\r', maxiter=400)
        ext = IdentityMatrix()+M.harmonic_extension
        inv = M.inner_solve + ext @ invS @ ext.T
        gfK.vec.data = inv*f.vec
        
        Draw(gfK, mesh, "K", min=0, max=1, deformation=True)
    
        errl2  = sqrt(Integrate( (gfK-K_ex)**2,mesh, order=max(5,2*(order+2)+1) ))
        errhm1 = CompHm1Norm(gfK-K_ex, order=order+3)
        
    return (errl2, errhm1, fesH.ndof)

We now test with a sequence of meshes and save the $L^2$ and $H^{-1}$ error. We perturb the mesh to avoid super-convergence due to mesh symmetry. Using the canonical interpolant instead of local $L^2$ projection + averaging gives one additional order of convergence.  In particular, if $g_h$ has polynomial degree $k$, then

\begin{align}
&\|K_h-K\|_{H^{-1}} \leq h^k\qquad\qquad \text{ for local $L^2$ projection}\\
&\|K_h-K\|_{H^{-1}} \leq h^{k+1}\qquad\quad \text{ for canonical interpolant}
\end{align}

[<a href="https://doi.org/10.1137/19M1255549">Gawlik: High-order approximation of Gaussian curvature with Regge finite elements, <i>SIAM J. Numerical Analysis</i> (2020).</a>]

[<a href="https://arxiv.org/abs/2206.09343">Gopalakrishnan, Neunteufel, Schöberl, Wardetzky:  Analysis of curvature approximations via covariant incompatibility and curl for Regge metrics, <i>arXiv</i> (2022).</a>]


In [ ]:
errhm1     = []
errl2      = []
ndof       = []

# order of metric approximation >= 0
order = 1

for i in range(6):
    print("i = ",i)
    ngmesh = unit_square.GenerateMesh(maxh=1/2**i)
    for pnts in ngmesh.Points():
        px,py = pnts[0],pnts[1]
        # perturb mesh points
        if abs(mapping(px,py)[0]+1) > 1e-8 and abs(mapping(px,py)[0]-1) > 1e-8 and abs(mapping(px,py)[1]+1) > 1e-8 and abs(mapping(px,py)[1]-1) > 1e-8:
            px += random.uniform(-1/2**(i+2),1/2**(i+2))
            py += random.uniform(-1/2**(i+2),1/2**(i+2))
        pnts[0] = mapping(px,py)[0]
        pnts[1] = mapping(px,py)[1]
    ngmesh.Update()
    mesh = Mesh(ngmesh)
    
    errl,errm1,dof = ComputeGaussCurv(mesh, Gex, order=order, can_int=False)
    errl2.append(errl)
    errhm1.append(errm1)
    ndof.append(dof)

Plot results.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import interactive

plt.plot(ndof, errhm1, '-o', label="$\|K_h-K_{\mathrm{ex}}\|_{H^{-1}}$")
plt.plot(ndof, errl2 , '-x', label="$\|K_h-K_{\mathrm{ex}}\|_{L^2}$")
plt.plot(ndof, [dof**(-1/2) for dof in ndof], '-' , color="k", label="$O(h)$")
plt.plot(ndof, [dof**(-2/2) for dof in ndof], '--', color="k", label="$O(h^2)$")
plt.plot(ndof, [dof**(-3/2) for dof in ndof], '-.', color="k", label="$O(h^3)$")

plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.show()
interactive(True)